In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/dataset_yolo.zip -d /content/

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/dataset_yolo/labels/train/72W1_jpg.rf.20498726429cb1896d2aeac492e37f8e.txt  
  inflating: /content/dataset_yolo/labels/train/72W2_jpg.rf.f4bb03963c17bd36aee133678dab3523.txt  
  inflating: /content/dataset_yolo/labels/train/72_jpg.rf.8b06e9486b2bef32e9df7cbd2310bd17.txt  
  inflating: /content/dataset_yolo/labels/train/72_jpg.rf.f4fc733cd655f66fc7cda89b6c807882.txt  
  inflating: /content/dataset_yolo/labels/train/72_jpg.rf.f9a02ef7bdc9b6b5582d3a8620b51d2e.txt  
  inflating: /content/dataset_yolo/labels/train/73C1_jpg.rf.a9ae2ba44fcfc3f7af345d6dc2aaacd1.txt  
  inflating: /content/dataset_yolo/labels/train/73C2_jpg.rf.e262f114d2c990f855f94810d3179d6d.txt  
  inflating: /content/dataset_yolo/labels/train/73W1_jpg.rf.cfe9cad3acbb1bc7a306df52bb2ab78b.txt  
  inflating: /content/dataset_yolo/labels/train/73W2_jpg.rf.3ae60d9da861919d65415fd37452846e.txt  
  inflating: /content/dataset_yolo/labels/train

In [ ]:
!pip install ultralytics -q

!nvidia-smi

import yaml
import shutil
from ultralytics import YOLO
from google.colab import drive
import os

with open('/content/dataset_yolo/data.yaml', 'r') as f:
    config = yaml.safe_load(f)
config['path'] = '/content/dataset_yolo'
with open('/content/dataset_yolo/data.yaml', 'w') as f:
    yaml.dump(config, f)

def save_to_drive_callback(trainer):
    epoch = trainer.epoch
    if epoch % 10 == 0:
        dst = '/content/drive/MyDrive/yolo_weights_backup'
        src = trainer.save_dir / 'weights'
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"💾 Эпоха {epoch}: веса сохранены в Drive!")

drive.mount('/content/drive')

model = YOLO('/content/drive/MyDrive/best.pt')

model.add_callback("on_train_epoch_end", save_to_drive_callback)

results = model.train(
    data='/content/dataset_yolo/data.yaml',
    epochs=150,
    imgsz=640,
    batch=24,
    device=0,
    project='power_line_fast',
    name='exp1',
    patience=15,
    save=True,
    resume=True,

    lr0=0.00005,
    lrf=0.01,

    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.01,

    optimizer='AdamW',
    momentum=0.937,
    weight_decay=0.0005,

    hsv_h=0.01,
    hsv_s=0.5,
    hsv_v=0.3,
    degrees=8.0,
    translate=0.05,
    scale=0.4,
    shear=1.5,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.3,

    mosaic=0.0,
    mixup=0.0,
    cutmix=0.0,
    copy_paste=0.0,

    dropout=0.0,
    erasing=0.0,

    freeze=15,
    close_mosaic=10,
    amp=True,
    save_period=10
)

print("✅ Обучение завершено!")

shutil.copy(
    'power_line_fast/exp1/weights/best.pt',
    '/content/drive/MyDrive/best_085_target.pt'
)

val_results = model.val(data='/content/dataset_yolo/data.yaml')
print(f"\n🎯 Финальный mAP@50: {val_results.box.map50:.4f}")
print(f"🎯 Финальный mAP@50-95: {val_results.box.map:.4f}")

if val_results.box.map50 >= 0.85:
    print("🎉 ЦЕЛЬ ДОСТИГНУТА! mAP@50 >= 0.85")
else:
    gap = 0.85 - val_results.box.map50
    print(f"⚠️  До цели осталось: {gap:.4f} mAP")


Thu Nov 20 23:10:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |   11848MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----